In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import datetime
import plotly.graph_objects as go
from ipywidgets import widgets, Layout
from IPython.display import display
from dash import Dash, dcc, html, Input, Output
import os
from threading import Timer
import webbrowser

In [ ]:
import warnings
warnings.simplefilter(action="ignore",category=FutureWarning)

In [ ]:
def open_browser():
    if not os.environ.get("WERKZEUG_RUN_MAIN"):
        webbrowser.open_new('http://127.0.0.1:8080/')

In [ ]:
linkedin_job_postings=pd.read_csv("..\Data\linkedin_jobs_and_skills\linkedin_job_postings_cleaned.csv")

In [ ]:
linkedin_job_postings.head()

In [ ]:
# Function to identify top hiring companies for a specific job position
def top_hiring_companies(jobs_df, country, top_n=9):
    
    # Filter jobs by country
    jobs_df = jobs_df[jobs_df['search_country'] == country]

    # Count occurrences of each company
    company_counts = jobs_df['company'].value_counts().head(top_n)
    
    return company_counts

In [ ]:
# Create initial plot
default_job_position = ''
default_country = 'United States'
initial_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(default_job_position.lower(), case=False)]
top_companies = top_hiring_companies(initial_jobs_df, default_country)

In [ ]:
fig = go.FigureWidget(
    data=[
        go.Bar(
            y=top_companies.index, 
            x=top_companies.values,
            text=top_companies.values,
            orientation='h',
        )
    ]
).update_layout(
    title=f"Top Companies hiring in \'{default_country}\'",
    yaxis_title="Company",
    xaxis_title="Number of Job Postings",
    yaxis=dict(autorange="reversed"),
    height=405,
)
fig.show()

In [ ]:
# Function to update plot based on filtered job position
def update_plot(search_value, country_value, top_n):
    if search_value:
        filtered_jobs_df = linkedin_job_postings[linkedin_job_postings['job_title'].str.contains(search_value.lower(), case=False)]
        if len(filtered_jobs_df) > 0:
            top_companies = top_hiring_companies(filtered_jobs_df, country_value, top_n)
            fig.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
            fig.data[0].x = top_companies.values
            fig.update_layout(
                title=f"Top companies hiring for \'{search_value}\' in \'{default_country}\'",
                yaxis=dict(autorange="reversed"),
                height = 400 if (top_n < 9) else (45 * top_n),
            )
        else:
            fig.data[0].y = []
            fig.data[0].x = []
            fig.update_layout(title=f"No matching job positions for {search_value}")
    else:
        top_companies = top_hiring_companies(linkedin_job_postings, country_value, top_n)
        fig.data[0].y = [(i[:30] + '..') if len(i) > 30 else i for i in top_companies.index]
        fig.data[0].x = top_companies.values
        fig.update_layout(
            title=f"Top companies hiring in \'{default_country}\'",
            yaxis=dict(autorange="reversed"),
            height = 400 if (top_n < 9) else (45 * top_n),
        )

In [ ]:
# Create search bar widget
# search_widget = widgets.Text(
#     value='',
#     placeholder='Search for a job position...',
#     description='Search:',
#     continuous_update=False,
#     layout=Layout(width='300px')
# )

# top_n_widget = widgets.IntSlider(
#     value=9,
#     description='Companies:',
#     min=1,
#     max=20,
#     continuous_update=False,
#     layout=Layout(width='400px')
# )

# def update_plot_interactive(search_value, top_n):
#     return update_plot(search_value, top_n)

# def on_search_button_clicked(b):
#     update_plot(search_widget.value, top_n_widget.value)

In [ ]:
# Create a button to trigger the search
# search_button = widgets.Button(description="Search")
# Attach the button click event handler
# search_button.on_click(on_search_button_clicked)

In [ ]:
# Display search bar and initial plot
# display(widgets.VBox([widgets.HBox([search_widget, top_n_widget, search_button]), fig]))

In [ ]:
# Create Dash app
app = Dash()

In [ ]:
# Define callback to update bar chart
@app.callback(
    Output('bar-chart', 'figure'),
    Input('search-button', 'n_clicks'),
    Input('search-input', 'value'),
    Input('country-input', 'value'),
    Input('top-n-input', 'value'),
)
def update_bar_chart(n_clicks, search_value, country_value, top_n):
    if n_clicks:
        update_plot(search_value, country_value, top_n)
    return fig

In [ ]:
# Define app layout
app.layout = html.Div([
    html.Div([
        dcc.Input(
            id='search-input',
            type='text',
            placeholder='Search for job titles...',
            debounce=True
        ),
        dcc.Dropdown(
            id='country-input',
            options=[{'label': i, 'value': i} for i in linkedin_job_postings['search_country'].unique()],
            value='United States',
            style={'width': '300px', 'height': '40px'},
        ),
        dcc.Input(
            id='top-n-input',
            type='number',
            placeholder='Top N Companies',
            min=1,
            max=20,
            value=9,
            debounce=True
        ),
        html.Button('Search', id='search-button')
    ]),
    html.Div([
        dcc.Graph(id='bar-chart')
    ])
])

In [ ]:
# app.run_server(debug=True, use_reloader=False)
Timer(1, open_browser).start()
app.run_server(debug=True, port=8080)